![](../images/rivacon_frontmark_combined_header.png)

# Equity Volatility Surfaces

In [ ]:
# import libs
import pyvacon
import matplotlib
matplotlib.use('nbagg')
import pandas as pd
import rivapy
from rivapy import marketdata as mkt_data
from rivapy import enums as enums
# import pyvacon
import datetime as dt
from dateutil.relativedelta import relativedelta
import math
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import plotly.graph_objects as go
import ipywidgets as widgets
from scipy.optimize import least_squares
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import warnings
warnings.filterwarnings('ignore')
#the next lin is a jupyter internal command to show the matplotlib graphs within the notebook
%matplotlib inline
#reload modules
%load_ext autoreload
%autoreload 2

## General Remarks
An equity volatility surface is an object providing for arbitrary expiries and strikes implied volatilities. The volatility surfaces provided by the analytics library are parametrized w.r.t. the so-called X-strikes, i.e. one has to put in a strike w.r.t. the X-variable which is the driving process of the spot $S$, i.e.
$$ S_t=(F_t-D_t)X_t+D_t$$
where $F_t$ is the risky forward and $D_t$ the cash dividends, see [Buehler](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1141877) for a more detailed discussion. 

To create a volatility surface one needs two components:
- A reference forward curve (typically the forward curve which was used to compute the implieds from quoted prices)
- A volatility parametrization



## Creating Forward Curve
We create a dummy forward curve as shown in the  [forward_curve](equity_forwardcurve.ipynb) notebook which will be used in all subsequent volatility surface constructions.

In [ ]:
refdate = dt.datetime(2021,10,8)

#dividend table neede fo forward curve
object_id = "TEST_DIV" 
ex_dates = [dt.datetime(2022,3,29), dt.datetime(2023,3,29), dt.datetime(2024,3,29), dt.datetime(2025,3,29)]
pay_dates = [dt.datetime(2022,4,1), dt.datetime(2023,4,1), dt.datetime(2024,4,1), dt.datetime(2025,4,1)]
tax_factors = [1.0, 1.0, 1.0, 1.0]
div_yield = [0, 0.005, 0.01, 0.01]
div_cash = [3.0, 2.0, 1.0, 0.0]
div_table = rivapy.marketdata.DividendTable(object_id, refdate, ex_dates, pay_dates, div_yield, div_cash, tax_factors)

#discount- and borrowing curve needed for forward curve
dates = [refdate + dt.timedelta(days=x) for x in [0,10]]
df = [1.0,1.0]
dc = mkt_data.DiscountCurve(object_id, refdate, dates, df, 
                             enums.InterpolationType.HAGAN_DF, enums.ExtrapolationType.NONE, enums.DayCounterType.Act365Fixed)
bc = mkt_data.DiscountCurve(object_id, refdate, dates, df, 
                             enums.InterpolationType.HAGAN_DF, enums.ExtrapolationType.NONE, enums.DayCounterType.Act365Fixed)
spot = 100.0

#forward curve
forward_curve = mkt_data.EquityForwardCurve(spot, dc, bc, div_table)

## Load Implied Vol Quotes

In [ ]:
# load sample equity option quote table
df_quote_table = pd.read_csv('../../marketdata/vol_quotes.csv',index_col=0)
df_quote_table

## Volatility Parametrizations

The volatility parametrization provides the method *calc_implied_vol* functionality to retrieve for each x-strike and time-to-maturity (in year fractions) the implied volatility. This method is used internally in the volatility surface in all methods where a implied volatility is computed.
We will discuss the different available parametrizations in this subsection.

### Flat Volatility
To setup a flat volatility, one may use the VolatilityParametrizationFlat

In [ ]:
flat_param = mkt_data.VolatilityParametrizationFlat(0.3)

### Term Structure Volatility
To create a volatility which has only a term structure and no strike dependency one may use this parametrization type. This parametrization needs a vector of expiry times (given as year fractions which are interpreted as year fractions w.r.t. the day counter specified in th volatility surface) and forward at-the-money volatilities, i.e. X-strike=1.0.

In [ ]:
ttm = [0.07,0.19,0.69,0.94,1.19,1.69]
fwd_atm_vols =  [0.25,0.24,0.22, 0.21, 0.20, 0.19]
term_param = mkt_data.VolatilityParametrizationTerm(ttm,fwd_atm_vols)

### SSVI
This parametrization is inspired by the volatility structure provided by stochastic volatility models. The total variance $w(k,t)$ for a strike log strike $k$ and time-to-maturity $t$ is given by
$$ w(k,t) = \frac{\theta_t}{2}\left( 1+\rho\phi(\theta_t)k+\sqrt{(\phi(\theta_t)k+\rho)^2+(1-\rho^2)}  \right) $$ 
and 
$$ \phi(\theta_t) = \frac{\eta}{\theta_t^\gamma(1+\theta_t)^{1-\gamma}} $$
for parameters $\rho$, $\eta$, $\gamma$ and given atm implied total variances $\theta_t:=\sigma^2(t)t$. The term structure of implied total variances is internally approximated by interpolation from given atm volatilities. The nice property of this surface is that there are very simple conditions on the parameters to guarantee that the surface is free of arbitrage, see [gatheral_jacquier_svi](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2033323).

In [ ]:
rho = -0.7 # responsible for the skewness of the vol_surface
eta = 0.7 # responsible for the curvature
gamma = 0.6 # responsible for the "rate of decay"
ssvi_param = mkt_data.VolatilityParametrizationSSVI(ttm, fwd_atm_vols, rho, eta, gamma)

### SVI

In [ ]:
svi_param = mkt_data.VolatilityParametrizationSVI(
                expiries=np.array([0.07,0.19,0.69,0.94,1.19,1.69]), 
                svi_params=[
                            (0.01, 0.08, -0.42, 0.07, 0.05),
                            (0.01, 0.09, -0.52, 0.08, 0.05),
                            (0.01, 0.09, -0.72, 0.09, 0.09),
                            (0.02, 0.08, -0.91, 0.10, 0.07),
                            (0.01, 0.12, -0.70, 0.11, 0.19),
                            (0.03, 0.09, -1.02, 0.13, 0.15)
])

In [ ]:
svi_param.calc_implied_vol(1/12,1)

### SABR

The [SABR](https://bsic.it/sabr-stochastic-volatility-model-volatility-smile/) model assumes that the forward rate and the instantaneous volatility are driven by two correlated Brownian motions:

$$df_t = \alpha_t f_t^\beta d W_t^1$$

$$d\alpha_t = \nu\alpha_t d W_t^2$$

$$E\bigl[d W_t^1 d W_T^2\bigr] = \rho d t$$

The expression that the implied volatility must satisfy is

$$\sigma_B(K,f) = \frac{\alpha\biggl\{1+\biggl[\frac{(1-\beta)^2}{24}\frac{\alpha^2}{(fK)^{1-\beta}}+\frac{1}{4}\frac{\rho\beta\nu\alpha}{(FK)^{(1-\beta)/2}}+\frac{2-3\rho^2}{24}\nu^2\biggr]T\biggr\}}{(fK)^{(1-\beta)/2}\biggl[1+\frac{(1-\beta)^2}{24}{ln}^2\frac{f}{K}+\frac{(1-\beta)^4}{1920}{ln}^4\frac{f}{K}\biggr]}\frac{z}{\chi(z)}$$

$$z=\frac{\nu}{\alpha}(fK)^{(1-\beta)/2}ln\frac{f}{K}$$

$$\chi(z) = ln\Biggl[\frac{\sqrt{1-2\rho z+z^2}+z-\rho}{1-\rho}\Biggr]$$

When $f = K $ (for ATM options), the above formula for implied volatility simplifies to:

$$\sigma_{ATM} = \sigma_B(f,f)=\frac{\alpha\biggl\{1+\biggl[\frac{(1-\beta)^2}{24}\frac{\alpha^2}{f^{2-2\beta}}+\frac{1}{4}\frac{\rho\beta\nu\alpha}{f^{1-\beta}}\frac{2-3\rho^2}{24}\nu^2\biggr]T\biggr\}}{f^{1-\beta}}$$

where

> $\alpha$ is the instantaneous vol;

> $\nu$ is the vol of vol;

> $\rho$ is the correlation between the Brownian motions driving the forward rate and the instantaneous vol;

> $\beta$ is the CEV component for forward rate (determines shape of forward rates, leverage effect and backbone of ATM vol).

For details please refer to the [SABR notebook](../models/sabr_volatility_model.ipynb). 

In [ ]:
ttm = [0.07,0.19,0.69,0.94,1.19,1.69]

sabr_params = np.array([[0.05, 0.90, 0.32, -0.82],
                        [0.08, 0.01, 1.22, -0.49],
                        [0.15, 0.20, 0.12, -0.79],
                        [0.18, 0.84, 0.58, -0.72],
                        [0.20, 0.62, 0.09, -0.58],
                        [0.20, 0.62, 0.09, -0.58]])

In [ ]:
sabr_param = mkt_data.VolatilityParametrizationSABR(ttm, sabr_params)
sabr_param.calc_implied_vol(1,1)

In [ ]:
# Plotting SABR Volatility Smile– Rivapy
# click "new plot" to create new plot, click "add trace" to add a trace with the current parameters

# create widgets
style = {'description_width': 'initial'}

FloatTextAlpha = widgets.FloatText(value = 0.15, step = 0.01, description = 'Alpha')

FloatSliderNu = widgets.FloatSlider(value = 0.8, min = 0.0001, max = 5, step = 0.01, description = 'Nu',
                                       continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f')

FloatSliderBeta = widgets.FloatSlider(value = 0.6, min = 0, max = 1, step = 0.01, description = 'Beta',
                                       continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f')

FloatSliderRho = widgets.FloatSlider(value = -0.7, min = -0.999999, max = 0.999999, step = 0.01, description = 'Rho',
                                       continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f')

FloatRangeSliderStrikes = widgets.FloatRangeSlider(value=[.4, 1.6], min=0, max=3.0, step=0.05, description='Strike Range:', 
                                                   disabled=False, continuous_update=False,orientation='horizontal',
                                                   readout=True, readout_format='.1f',style=style)

FloatSliderExpiries = widgets.FloatSlider(value=1, min=1/12, max=30.0, step=1/12, description='Expiry:', 
                                                   disabled=False, continuous_update=False,orientation='horizontal',
                                                   readout=True, readout_format='.2f',style=style)

ButtonNewPlot1 = widgets.Button(description="New Plot")

ButtonAddTrace1 = widgets.Button(description="Add Trace")

global OutputWidget
OutputWidget = widgets.Output()

def create_vol_grid(alpha, nu, beta, rho,strike_range, expiry):
    F_0 = 1
    strikes = np.linspace(strike_range[0], strike_range[1], num=100)
    sabr_params = np.array([[alpha,nu,beta,rho]])
    ttm = [expiry]
    sabr_param = mkt_data.VolatilityParametrizationSABR(ttm, sabr_params)
    vols = [sabr_param.calc_implied_vol(expiry,x) for x in strikes]

    return strikes, vols

def create_plot(strikes, vols,expiry,alpha, nu, beta, rho):    
    fig1.add_trace(go.Scatter(x= strikes,y= vols 
                            ,mode = 'lines+markers'
                          ,hovertemplate = 
                            'Moneyness:  %{x: .1%}' #+\
                            +'<br>Volatility: %{y: .1%}'
                            +'<br>Expiry: {:,.0f} Yrs'.format(expiry)
                            +'<br>Alpha: {:,.1%}'.format(alpha)
                            +'<br>Nu: {:,.1f}'.format(nu)
                            +'<br>Beta: {:,.1f}'.format(beta)
                            +'<br>Rho: {:,.1f}'.format(rho)
                            +'<extra></extra>',
                            showlegend=False)
                         )


    fig1.update_layout(title={
                          'text': "<b>Volatility Smile</b>",
                          'y':0.95,
                          'x':0.5,
                          'xanchor': 'center',
                          'yanchor': 'top'
                            }
                    ,width=1000
                    ,height=500
                    ,xaxis_title='Moneyness'
                    ,xaxis_tickformat = '.1%'
                    ,xaxis_range=[strikes.min(),strikes.max()]
                    ,yaxis_title='Volatility'
                    ,yaxis_tickformat = '.1%'
                    ,yaxis_range=[0,1]
                    ,font=dict(
                      family="Courier New, monospace"
                      ,size=10
                      )
                    ,margin=dict(l=65, r=50, b=65, t=90)
    )
    fig1.show()

def plot(alpha, nu, beta, rho,strike_range, expiry):
#     function is called by eventhandler, i.e. if input parameter changes

#     clear output
    OutputWidget.clear_output()
    
#     1. create vol grid
    strikes, vols = create_vol_grid(alpha, nu, beta, rho,strike_range, expiry)

#     2. plot surface
    create_plot(strikes, vols,expiry,alpha, nu, beta, rho)
    

def eventhandler(change):
          
    alpha = FloatTextAlpha.value
    nu = FloatSliderNu.value
    beta = FloatSliderBeta.value
    rho = FloatSliderRho.value
    strike_range = FloatRangeSliderStrikes.value
    expiry = FloatSliderExpiries.value
    
#     call plot function
    with OutputWidget:
        plot(alpha, nu, beta, rho,strike_range, expiry)
        
def eventhandler2(change):
    global fig1
    
    fig1 = go.Figure()
    
    alpha = FloatTextAlpha.value
    nu = FloatSliderNu.value
    beta = FloatSliderBeta.value
    rho = FloatSliderRho.value
    strike_range = FloatRangeSliderStrikes.value
    expiry = FloatSliderExpiries.value
    
    with OutputWidget:
        plot(alpha, nu, beta, rho,strike_range, expiry)

# bind eventhandler to widgets
ButtonAddTrace1.on_click(eventhandler)
ButtonNewPlot1.on_click(eventhandler2)

# widgets groups
WidgetsGrpH1 = widgets.HBox(children=[widgets.Label('Set Chart Area:')])
WidgetsGrpH2 = widgets.HBox(children=[FloatRangeSliderStrikes])
WidgetsGrpH3 = widgets.HBox(children=[widgets.Label('Set Parameters:')])
WidgetsGrpH4 = widgets.HBox(children=[FloatTextAlpha,FloatSliderExpiries])
WidgetsGrpH5 = widgets.HBox(children=[FloatSliderNu,FloatSliderBeta,FloatSliderRho])
WidgetsGrpH6 = widgets.HBox(children=[ButtonNewPlot1,ButtonAddTrace1])
WidgetsGrpV1 = widgets.VBox(children=[WidgetsGrpH1,WidgetsGrpH2,WidgetsGrpH3,WidgetsGrpH4,WidgetsGrpH5,WidgetsGrpH6])

display(WidgetsGrpV1)
display(OutputWidget)

In [ ]:
# Build App
# Figure shows quotes of closest expiry
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__,external_stylesheets=external_stylesheets)
# app = JupyterDash(__name__)

app.layout = html.Div([
    
#     html.Div([html.H1('Volatility Smile', style={'text-align': 'center'})]),
    
    html.Div([
    
        html.Div([

            html.Label(['Nu',        
                dcc.Slider(id = 'nu_slider', value = 0.7, min = 0.0001, max = 5, step = 0.01,updatemode='drag',
                          marks={i: '{:,.1f}'.format(i) for i in np.arange(0, 5.5, 0.5).tolist()},included=False)
                       ]),
            
            html.Div(id='nu_slider_out',style={'margin-top':20}),
            
            html.Br(),

            html.Label(['Beta',        
                dcc.Slider(id = 'beta_slider', value = 0.6, min = 0, max = 1, step = 0.01,updatemode='drag',
                              marks={i: '{:,.1f}'.format(i) for i in np.arange(0, 1.25, 0.25).tolist()},included=False)
                       ]),
            
            html.Div(id='beta_slider_out',style={'margin-top':20}),
            
            html.Br(),

            html.Label(['Rho',        
                dcc.Slider(id = 'rho_slider', value = -0.7, min = -0.999999, max = 0.999999, step = 0.01,updatemode='drag',
                              marks={i: '{:,.1f}'.format(i) for i in np.arange(-1, 1.25, 0.25).tolist()},included=False)
                       ]),

            html.Div(id='rho_slider_out',style={'margin-top':20}),

        ],style={'width': '40%', 'height':'30%', 'display': 'inline-block', 'vertical-align': 'top','margin-left':20}),

        html.Div([

            html.Label(['Strike Range',        
                dcc.RangeSlider(id = 'strike_range_slider', count=1, min = 0, max = 3.0, step = 0.05, value = [.4, 1.6],
                              marks={i: '{}'.format(i) for i in np.arange(0, 3.5, 0.5).tolist()})
                       ],),
            
            html.Div(id='strike_range_slider_out',style={'margin-top':20}),
            
            html.Br(),


            html.Label(['Expiry (Yrs.)',        
                dcc.Slider(id = 'expiry_slider', min = 0, max = 5.0, step = 0.01, value = .94, updatemode='drag',

                              marks={i: '{:,.0f}'.format(i) for i in np.arange(0, 6, 1).tolist()},included=False)
                          ]),
            
            html.Div(id='expiry_slider_out',style={'margin-top':20}),
            
            html.Br(),

            html.Label(['Alpha',        
                dcc.Input(id = 'alpha_input', min = 0.00000, value = .15, step = .01, type = 'number')
                       ])

        ],style={'width': '40%','height':'30%', 'display': 'inline-block', 'vertical-align': 'top','margin-left':20}),
    
    ],style={'font-family':'Courier New, monospace','font-size':15}),
        
#     html.Br(),
    
    html.Div([
        dcc.Graph(id='vol_smile',figure = {}), 
            ],style={}),
    
    
])

# Define callback to update graph
@app.callback(
    [Output(component_id='nu_slider_out', component_property='children'),
     Output(component_id='beta_slider_out', component_property='children'),
     Output(component_id='rho_slider_out', component_property='children'),
     Output(component_id='strike_range_slider_out', component_property='children'),
     Output(component_id='expiry_slider_out', component_property='children'),
    Output(component_id='vol_smile', component_property='figure')]
    
    ,
    [
    Input(component_id='strike_range_slider', component_property='value'),
    Input(component_id='alpha_input', component_property='value'),
    Input(component_id='nu_slider', component_property='value'),
    Input(component_id='beta_slider', component_property='value'),
    Input(component_id='rho_slider', component_property='value'),
    Input(component_id='expiry_slider', component_property='value')
    ]
)


def update_graph(strike_range,alpha,nu,beta,rho,expiry):#, sigma_0, alpha, beta, rho, expiry):
   
    df_quotes = df_quote_table.copy()
    df_quotes['dif'] = (df_quotes['EXPIRY'] - expiry).abs()
    df_quotes = df_quotes[df_quotes['dif'] == df_quotes['dif'].min()]

    F_0 = 1
    strikes = np.linspace(strike_range[0], strike_range[1], num=100)
    sabr_params = np.array([[alpha,nu,beta,rho]])
    ttm = [df_quotes['dif'].min()]
    sabr_param = mkt_data.VolatilityParametrizationSABR(ttm, sabr_params)
    vols = [sabr_param.calc_implied_vol(expiry,x) for x in strikes]
    


    fig = go.Figure(data=[go.Scatter(x= strikes,y= vols 
                            ,mode = 'lines+markers'
                            ,name = 'SABR Vol'
                          ,hovertemplate = 
                            'Moneyness:  %{x: .1%}' #+\
                            +'<br>Volatility: %{y: .1%}'
                            +'<br>Expiry: {:,.0f} Yrs'.format(expiry)
                            +'<br>Alpha: {:,.1%}'.format(alpha)
                            +'<br>Nu: {:,.1f}'.format(nu)
                            +'<br>Beta: {:,.1f}'.format(beta)
                            +'<br>Rho: {:,.1f}'.format(rho)
                            +'<extra></extra>',
                            showlegend=True)
#                             +'<br>Volatility: %{y: .1%}<extra></extra>')
#                          ,colorscale = 'temps')
                         ])
    
    
    fig.add_trace(go.Scatter(x=df_quotes.STRIKE, 
                                   y= df_quotes.ASK_IV, 
                                   mode = 'markers',
                                   marker_symbol=106,
                                   marker_color='rgb(11, 83, 148)',
                                   name = 'Ask Quotes',
                                  hovertemplate = 
                                    '<br>Ask Quote'
                                    +'<br>Moneyness:  %{x: .1%}' #+\
                                    +'<br>Volatility: %{y: .1%}'
                                    +'<br>Expiry: {:,.2f} Yrs'.format(df_quotes.EXPIRY.values[0])
                                    +'<extra></extra>',
                                  showlegend=True)

                       )

    fig.add_trace(go.Scatter(x=df_quotes.STRIKE, 
                                   y= df_quotes.BID_IV, 
                                   mode = 'markers',
                                   marker_symbol=105,
                                   marker_color='rgb(204,0,0)',
                                   name = 'Bid Quotes',
                                  hovertemplate = 
                                    '<br>Bid Quote'
                                    +'<br>Moneyness:  %{x: .1%}' #+\
                                    +'<br>Volatility: %{y: .1%}'
                                    +'<br>Expiry: {:,.2f} Yrs'.format(df_quotes.EXPIRY.values[0])
                                    +'<extra></extra>',
                                  showlegend=True)

                       )


    fig.update_layout(title={
                          'text': "<b>Volatility Smile</b>",
                          'y':0.9,
                          'x':0.5,
                          'xanchor': 'center',
                          'yanchor': 'top'
                            }
#                     ,autosize=True
                    ,width=900
                    ,height=500
                    ,xaxis_title='Moneyness'
                    ,xaxis_tickformat = '.1%'
                    ,xaxis_range=[strikes.min(),strikes.max()]
    #                     ,xaxis_autorange = 'reversed'
                    ,yaxis_title='Volatility'
                    ,yaxis_tickformat = '.1%'
                    ,yaxis_range=[0,1]
                    ,font=dict(
                      family="Courier New, monospace"
                      ,size=10
                      )
                    ,margin=dict(l=65, r=50, b=65, t=90)
    )

    return 'Nu: {}'.format(nu),'Beta: {}'.format(beta),'Rho: {}'.format(rho),'Range: {}-{}'.format(strike_range[0],strike_range[1]), 'Expiry: {}'.format(expiry), fig

# Run app and display result inline in the notebook
# app.run_server(mode='inline')
app.run_server()

## Volatility Smile

### Plot Parametrization

In [ ]:
# compare parametrizations
vol_params_dict = {'Flat': flat_param
                   ,'Term': term_param
                   ,'SVI': svi_param
                   ,'SSVI': ssvi_param
                   ,'SABR': sabr_param
                  }

In [ ]:
# Comparing Volatility Smiles – Rivapy
# click "new plot" to create new plot, click "add trace" to add a trace with the current parameters

# create widgets
style = {'description_width': 'initial'}

keys_list = list(vol_params_dict.keys())
SelectMultipleParams = widgets.SelectMultiple(options=keys_list,value=[keys_list[0]],description='Parametrization',
                                                disabled=False, continuous_update=False,readout=True,style=style)

FloatRangeSliderStrikes = widgets.FloatRangeSlider(value=[.4, 1.6], min=0, max=3.0, step=0.05, description='Strike Range', 
                                                   disabled=False, continuous_update=False,orientation='horizontal',
                                                   readout=True, readout_format='.1f',style=style)

DatePickerRefDate = widgets.DatePicker(description='Ref Date',value = dt.date(2022,1,1), disabled=False, continuous_update=False,
                                                   readout=True,style=style)

DatePickerExpiryDate = widgets.DatePicker(description='Expiry Date',value = dt.date(2023,1,1), disabled=False, continuous_update=False,
                                                   readout=True,style=style)


ButtonNewPlot2 = widgets.Button(description="New Plot")

ButtonAddTrace2 = widgets.Button(description="Add Trace(s)")

global OutputWidget2
OutputWidget2 = widgets.Output()


def create_plot2(param_list,strike_range, ref_date, expiry_date):    
    
    moneyness = np.linspace(strike_range[0], strike_range[1], num=100)
    strikes = np.array([i*forward_curve.value(ref_date,expiry_date) for i in moneyness])
    expiry = (expiry_date-ref_date).days/365.25
    
    for p in param_list:
        param = vol_params_dict[p]
        vol_surf = mkt_data.VolatilitySurface(p+'_surf', ref_date, forward_curve, enums.DayCounterType.Act365Fixed, param)
        vols = [vol_surf.calc_implied_vol(expiry_date,x,ref_date) for x in strikes]        
    
        fig2_smile.add_trace(go.Scatter(x= moneyness,y= vols 
                                ,mode = 'lines+markers'
                              ,hovertemplate =
                                '<br>Parametrization: {}'.format(p)
                                +'<br>Moneyness:  %{x: .1%}' #+\
                                +'<br>Volatility: %{y: .1%}'
                                +'<br>Expiry: {:,.2f} Yrs'.format(expiry)
                                +'<extra></extra>',
                                showlegend=False)
                             )


    fig2_smile.update_layout(title={
                          'text': "<b>Volatility Smile</b>",
                          'y':0.95,
                          'x':0.5,
                          'xanchor': 'center',
                          'yanchor': 'top'
                            }
                    ,width=1000
                    ,height=500
                    ,xaxis_title='Moneyness'
                    ,xaxis_tickformat = '.1%'
                    ,xaxis_range=[moneyness.min(),moneyness.max()]
                    ,yaxis_title='Volatility'
                    ,yaxis_tickformat = '.1%'
                    ,yaxis_range=[0,1]
                    ,font=dict(
                      family="Courier New, monospace"
                      ,size=10
                      )
                    ,margin=dict(l=65, r=50, b=65, t=90)
    )
    fig2_smile.show()

def plot2(param_list,strike_range, ref_date, expiry_date):
#     function is called by eventhandler, i.e. if input parameter changes

#     clear output
    OutputWidget2.clear_output()

#     plot surface
    create_plot2(param_list,strike_range, ref_date, expiry_date)
    

def eventhandler3(change):
          
    param_list = list(SelectMultipleParams.value)
    strike_range = FloatRangeSliderStrikes.value
    ref_date = dt.datetime.combine(DatePickerRefDate.value, dt.datetime.min.time())
    expiry_date = dt.datetime.combine(DatePickerExpiryDate.value, dt.datetime.min.time())
    
#     call plot function
    with OutputWidget2:
        plot2(param_list,strike_range, ref_date, expiry_date)
        
def eventhandler4(change):
    global fig2_smile
    
    fig2_smile = go.Figure()
    
    param_list = list(SelectMultipleParams.value)
    strike_range = FloatRangeSliderStrikes.value
    ref_date = dt.datetime.combine(DatePickerRefDate.value, dt.datetime.min.time())
    expiry_date = dt.datetime.combine(DatePickerExpiryDate.value, dt.datetime.min.time())
    
    with OutputWidget2:
        plot2(param_list,strike_range, ref_date, expiry_date)

# bind eventhandler to widgets
ButtonAddTrace2.on_click(eventhandler3)
ButtonNewPlot2.on_click(eventhandler4)

# widgets groups
WidgetsGrpV1 = widgets.VBox(children=[widgets.Label('Set Chart Area:'),FloatRangeSliderStrikes])
WidgetsGrpV2 = widgets.VBox(children=[widgets.Label('Set Reference and Expiry Date:'),DatePickerRefDate,DatePickerExpiryDate])
WidgetsGrpV3 = widgets.VBox(children=[widgets.Label('Select Parametrization(s):'),SelectMultipleParams])
WidgetsGrpH1 = widgets.HBox(children=[WidgetsGrpV1,WidgetsGrpV2,WidgetsGrpV3])
WidgetsGrpH2 = widgets.HBox(children=[ButtonNewPlot2,ButtonAddTrace2])

display(WidgetsGrpH1,WidgetsGrpH2)
display(OutputWidget2)

### Fitting Parametrizations

To illustrate the fitting of the parameters of the different volatility parametrizations shown, a simple calibration function is set up below. It is also implemented as method for the SVI and SABR parametrizations in rivapy.

In [ ]:
def calibrate(vol_param, quotes):
    def cost_function(x):
        vol_param._set_param(x)
        quotes['VOLS'] = [vol_param.calc_implied_vol(expiry,strike) for expiry, strike in zip(quotes['EXPIRY'],quotes['STRIKE'])]
        quotes['DIST_ASK']  = [max(vol-ask,0) for ask, vol in zip(quotes['ASK_IV'],quotes['VOLS'])]
        quotes['DIST_BID']  = [max(bid-vol,0) for bid, vol in zip(quotes['BID_IV'],quotes['VOLS'])]
        quotes['DIST_TOTAL'] = quotes['DIST_ASK']+quotes['DIST_BID']
        return np.copy(quotes['DIST_TOTAL'].values)
        
    result = least_squares(cost_function,vol_param._x,method='lm',diff_step=1e-3)
    
    return result.x

# calibrate(sabr_param,df_quote_table)

In [ ]:
# calibrate sabr parameters using rivapy
sabr_param.calibrate_params(df_quote_table,method = 'lm')

In [ ]:
# calibrate svi parameters using rivapy
svi_param.calibrate_params(df_quote_table,method = 'lm')

In [ ]:
# calibrate ssvi parameters using function given above
ssvi_param._set_param(calibrate(ssvi_param,df_quote_table))

In [ ]:
# plot fitted parametrizations

expiries = np.array([0.07,0.19,0.69,0.94,1.19,1.69])


style = {'description_width': 'initial'}
DropdownWidget = widgets.Dropdown(options = expiries, description = 'Select Expiry:',disabled = False,continuous_update=False,orientation='horizontal',
                                                   readout=True, readout_format='.2f',style=style)

ButtonNewPlot3 = widgets.Button(description="New Plot")

global OutputWidget3
OutputWidget3 = widgets.Output()   

def plot3(exp):
    
    ref_date = dt.datetime(2021,10,9)
    moneyness = np.linspace(.5, 1.5, num=100)
    expiries = np.array([0.07,0.19,0.69,0.94,1.19,1.69])
    idx = list(expiries).index(exp)
    
    expiry_date = ref_date + dt.timedelta(days=expiries[idx]*365)
    strikes = np.array([i*forward_curve.value(ref_date,expiry_date) for i in moneyness])
    df_quotes = df_quote_table[df_quote_table.EXPIRY == df_quote_table.EXPIRY.unique()[idx]].copy()


    vol_surf_sabr = mkt_data.VolatilitySurface('sabr_surf', ref_date, forward_curve, enums.DayCounterType.Act365Fixed, sabr_param)
    vols_sabr = [vol_surf_sabr.calc_implied_vol(expiry_date,x,ref_date) for x in strikes] 

    vol_surf_svi = mkt_data.VolatilitySurface('sabr_surf', ref_date, forward_curve, enums.DayCounterType.Act365Fixed, svi_param)
    vols_svi = [vol_surf_svi.calc_implied_vol(expiry_date,x,ref_date) for x in strikes]     

    vol_surf_ssvi = mkt_data.VolatilitySurface('sabr_surf', ref_date, forward_curve, enums.DayCounterType.Act365Fixed, ssvi_param)
    vols_ssvi = [vol_surf_ssvi.calc_implied_vol(expiry_date,x,ref_date) for x in strikes] 
    
    
    # Create Figure
    fig_smile = go.Figure()

    fig_smile.add_trace(go.Scatter(x= moneyness,y= vols_sabr 
                            ,mode = 'lines+markers'
                            ,name = 'SABR'
                          ,hovertemplate =
                            '<br>Parametrization: SABR'
                            +'<br>Moneyness:  %{x: .1%}' #+\
                            +'<br>Volatility: %{y: .1%}'
                            +'<br>EXPIRY: {:,.2f} Yrs'.format(df_quotes.EXPIRY.values[0])
                            +'<extra></extra>',
                            showlegend=True,
                            visible='legendonly')
                         )

    fig_smile.add_trace(go.Scatter(x= moneyness,y= vols_svi 
                            ,mode = 'lines+markers'
                            ,name = 'SVI'
                          ,hovertemplate =
                            '<br>Parametrization: SVI'
                            +'<br>Moneyness:  %{x: .1%}' #+\
                            +'<br>Volatility: %{y: .1%}'
                            +'<br>EXPIRY: {:,.2f} Yrs'.format(df_quotes.EXPIRY.values[0])
                            +'<extra></extra>',
                            showlegend=True,
                            visible='legendonly')
                         )

    fig_smile.add_trace(go.Scatter(x= moneyness,y= vols_ssvi 
                            ,mode = 'lines+markers'
                            ,name = 'SSVI'
                          ,hovertemplate =
                            '<br>Parametrization: SSVI'
                            +'<br>Moneyness:  %{x: .1%}' #+\
                            +'<br>Volatility: %{y: .1%}'
                            +'<br>EXPIRY: {:,.2f} Yrs'.format(df_quotes.EXPIRY.values[0])
                            +'<extra></extra>',
                            showlegend=True,
                            visible='legendonly')
                         )

    fig_smile.add_trace(go.Scatter(x=df_quotes.STRIKE, 
                                   y= df_quotes.ASK_IV, 
                                   mode = 'markers',
                                   marker_symbol=106,
                                   marker_color='rgb(11, 83, 148)',
                                   name = 'Ask Quotes',
                                  hovertemplate = 
                                    '<br>Ask Quote'
                                    +'<br>Moneyness:  %{x: .1%}' #+\
                                    +'<br>Volatility: %{y: .1%}'
                                    +'<br>EXPIRY: {:,.2f} Yrs'.format(df_quotes.EXPIRY.values[0])
                                    +'<extra></extra>',
                                  showlegend=True)

                       )

    fig_smile.add_trace(go.Scatter(x=df_quotes.STRIKE, 
                                   y= df_quotes.BID_IV, 
                                   mode = 'markers',
                                   marker_symbol=105,
                                   marker_color='rgb(204,0,0)',
                                   name = 'Bid Quotes',
                                  hovertemplate = 
                                    '<br>Bid Quote'
                                    +'<br>Moneyness:  %{x: .1%}' #+\
                                    +'<br>Volatility: %{y: .1%}'
                                    +'<br>EXPIRY: {:,.2f} Yrs'.format(df_quotes.EXPIRY.values[0])
                                    +'<extra></extra>',
                                  showlegend=True)

                       )


    fig_smile.update_layout(title={
                          'text': "<b>Volatility Smile</b>",
                          'y':0.95,
                          'x':0.5,
                          'xanchor': 'center',
                          'yanchor': 'top'
                            }
                    ,width=1000
                    ,height=500
                    ,xaxis_title='Moneyness'
                    ,xaxis_tickformat = '.1%'
                    ,xaxis_range=[moneyness.min(),moneyness.max()]
                    ,yaxis_title='Volatility'
                    ,yaxis_tickformat = '.1%'
                    ,yaxis_range=[.1,.4]
                    ,font=dict(
                      family="Courier New, monospace"
                      ,size=10
                      )
                    ,margin=dict(l=65, r=50, b=65, t=90)
    )
    fig_smile.show()

def eventhandler5(change):
    
    OutputWidget3.clear_output()      
        
    exp = DropdownWidget.value
    
#     call plot function
    with OutputWidget3:
        plot3(exp)

def eventhandler6(change):
    
    OutputWidget3.clear_output()      
        
    exp = DropdownWidget.value
    
#     call plot function
    with OutputWidget3:
        plot3(exp)

ButtonNewPlot3.on_click(eventhandler6)        
DropdownWidget.observe(eventhandler5, names='value')

display(ButtonNewPlot3)
display(DropdownWidget)
display(OutputWidget3)

## Volatility Surface
The forward curve and the parametrization can now be combined into a VolatilitySurface

In [ ]:
obj_id = 'TEST_SURFACE'
vol_surf = mkt_data.VolatilitySurface(obj_id, refdate, forward_curve, enums.DayCounterType.Act365Fixed, sabr_param)

To compute an implied volatility, one may use the method *calcImpliedVol*. Note that this method applies a sticky-strike handling of volatilities, i.e. it assumes that the implied volatility given a certain strike is independent of current forward values which may differ from the forwards when the volatility surface was calibrated.

In [ ]:
vol = vol_surf.calc_implied_vol(refdate + dt.timedelta(days=180),120,refdate)
print(vol)

By executing the following command line, the volatility surface is plotted.

In [ ]:
# plot surface
refdate = dt.datetime(2021, 10, 9)
expiries_surf = [refdate + dt.timedelta(days=expiries[0]*365),
    refdate + dt.timedelta(days=expiries[1]*365),
    refdate + dt.timedelta(days=expiries[2]*365),
    refdate + dt.timedelta(days=expiries[3]*365),
    refdate + dt.timedelta(days=expiries[4]*365),
    refdate + dt.timedelta(days=expiries[5]*365),
]

# strikes = list(s_range(80, 120, 100))
moneyness = np.linspace(0.5, 1.5, 100)

y = moneyness
x = ttm

term_structure = []
for i in moneyness:
    temp = []
    for j in expiries_surf:
        strike = i * forward_curve.value(refdate, j)
        temp.append(vol_surf.calc_implied_vol(j, strike, refdate))
    term_structure.append(temp)

fig_surf = go.Figure(data=[go.Surface(x=x, y=y,z=term_structure
                        ,contours = {"x": {"show": True,"size": 0.1, "color":"red"},
                                    "y": {"show": True,"size": 0.1, "color":"red"},}
                        ,opacity = .75
                        ,hovertemplate =
                        'Moneyness:  %{y: .2%}' +\
                        '<br>Maturity (yrs): %{x: .1f}' +\
                        '<br>Volatility: %{z: .2f}<extra></extra>'
                     ,colorscale = 'temps')
                     ])

fig_surf.update_layout(
    title={
        'text': "<b>Volatility Surface</b>",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }
    # ,autosize=True
    ,
    width=1000,
    height=500,
    scene=dict(xaxis_title='Maturity (yrs)',
               xaxis_tickformat='.1f',
               xaxis_autorange='reversed',
               yaxis_title='Moneyness',
               yaxis_tickformat='.2%',
               zaxis_title='Volatility',
               zaxis_tickformat='.2%'),
    font=dict(family="Courier New, monospace", size=10),
    margin=dict(l=65, r=50, b=65, t=90))

fig_surf.show()

---